In [136]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [137]:
#This notebook requires the following modules
import os
import numpy
import csv

In [138]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [139]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [140]:
events.count('2756') #catch trials

7

In [141]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [142]:
#uberlist

In [143]:
'2810' in uberlist[74][2]

False

In [144]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [145]:
#RT

In [146]:
#The following provides indexed lists for each of the individual event codes of interest 

In [147]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [148]:
#float(times[74])-float(times[70])

In [149]:
#Saccade

In [150]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [151]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [152]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [153]:
#print(TimeTrials[0])

In [154]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [155]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [156]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [157]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [158]:
#RT

In [159]:
checksum = sum(error)+sum(abort)+sum(catch)

In [160]:
checksum

588

In [161]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 0 0 0 0 0 0 90
2 1 0 0 0 0 0 225
3 0 1 0 1 0 210.0017000000007 225
4 1 0 0 0 0 0 90
5 1 0 0 0 0 0 315
6 0 1 0 1 0 195.99369999999908 315
7 1 0 0 0 0 0 270
8 0 1 0 1 0 210.0018999999993 135
9 0 1 0 1 0 223.96919999999955 270
10 1 0 0 0 0 0 45
11 1 0 0 0 0 0 180
12 0 1 0 1 0 204.02160000000003 180
13 1 0 0 0 0 0 90
14 0 1 0 1 0 159.98959999999715 225
15 0 1 0 1 0 198.00069999999687 270
16 1 0 0 0 0 0 315
17 1 0 0 0 0 0 90
18 1 0 0 0 0 0 180
19 0 1 0 1 0 178.01209999999992 45
20 1 0 0 0 0 0 0
21 1 0 0 0 0 0 270
22 0 1 0 1 0 204.0216999999975 90
23 0 1 0 1 0 173.9161000000022 90
24 1 0 0 0 0 0 225
25 1 0 0 0 0 0 315
26 1 0 0 0 0 0 180
27 0 0 0 1 0 164.0037999999986 270
28 1 0 0 0 0 0 135
29 1 0 0 0 0 0 0
30 0 1 0 1 0 198.00070000000414 315
31 1 0 0 0 0 0 225
32 1 0 0 0 0 0 270
33 0 1 0 1 0 156.01660000000265 180
34 1 0 0 0 0 0 270
35 0 1 0 1 0 201.97370000000228 180
36 1 0 0 0 0 0 270
37 0 1 0 1 0 185.9991000000009 135
38 1 0 0 0 0 0 225
39 0 1 0 1 0 140.00130000000354 315
40 1 0 0 0 0 0

501 1 0 0 0 0 0 135
502 0 1 0 1 0 202.01470000005793 225
503 1 0 0 0 0 0 135
504 1 0 0 0 0 0 135
505 0 1 0 1 0 171.99110000010114 135
506 1 0 0 0 0 0 315
507 0 1 0 1 0 226.01740000001155 225
508 1 0 0 0 0 0 270
509 1 0 0 0 0 0 270
510 1 0 0 0 0 0 225
511 1 0 0 0 0 0 315
512 1 0 0 0 0 0 225
513 0 1 0 1 0 212.00920000008773 225
514 0 0 0 0 1 0 90
515 1 0 0 0 0 0 90
516 1 0 0 0 0 0 180
517 1 0 0 0 0 0 180
518 0 1 0 1 0 294.01089999999385 225
519 1 0 0 0 0 0 270
520 1 0 0 0 0 0 90
521 0 1 0 1 0 188.00639999995474 225
522 0 1 0 1 0 200.00769999995828 0
523 0 0 1 0 0 231.99750000005588 225
524 0 1 0 1 0 273.98139999993145 225
525 1 0 0 0 0 0 270
526 1 0 0 0 0 0 270
527 1 0 0 0 0 0 45
528 0 1 0 1 0 303.9641999999294 315
529 0 0 0 0 1 0 225
530 0 1 0 1 0 224.0102000000188 180
531 0 0 0 1 0 271.97440000006463 270
532 1 0 0 0 0 0 180
533 0 1 0 1 0 207.9948999999324 180
534 1 0 0 0 0 0 135
535 1 0 0 0 0 0 225
536 1 0 0 0 0 0 180
537 0 1 0 1 0 240.02549999998882 90
538 0 0 0 1 0 53.98540000000503 

807 1 0 0 0 0 0 270
808 0 0 0 1 0 224.01029999996535 45
809 0 1 0 1 0 227.98340000002645 135
810 0 1 0 1 0 210.00190000003204 135
811 0 1 0 1 0 202.01459999987856 270
812 0 1 0 1 0 202.01490000006743 270
813 1 0 0 0 0 0 270
814 1 0 0 0 0 0 90
815 1 0 0 0 0 0 225
816 0 1 0 1 0 212.00900000007823 0
817 0 1 0 1 0 216.02289999998175 180
818 0 1 0 1 0 195.9936999999918 0
819 0 0 0 1 0 211.96819999977015 180
820 0 0 0 1 0 198.00059999991208 225
821 0 0 1 0 0 925.9827000000514 90
822 1 0 0 0 0 0 270
823 1 0 0 0 0 0 90
824 0 1 0 1 0 229.99040000000969 0
825 0 1 0 1 0 169.98389999987558 0
826 0 1 0 1 0 203.98090000008233 45
827 1 0 0 0 0 0 180
828 0 1 0 1 0 207.99489999981597 270
829 1 0 0 0 0 0 0
830 0 1 0 1 0 323.99359999992885 270
831 0 1 0 1 0 235.9704999998212 315
832 1 0 0 0 0 0 315
833 0 1 0 1 0 237.9777999999933 270
834 0 1 0 1 0 243.99869999988005 90
835 0 1 0 1 0 176.00510000018403 180
836 1 0 0 0 0 0 180
837 1 0 0 0 0 0 45
838 1 0 0 0 0 0 270
839 0 1 0 1 0 323.91170000005513 315
840 

1124 1 0 0 0 0 0 90
1125 0 1 0 1 0 198.00069999997504 180
1126 1 0 0 0 0 0 180
1127 1 0 0 0 0 0 180
1128 1 0 0 0 0 0 270
1129 0 1 0 1 0 205.9879999998957 315
1130 0 1 0 1 0 219.99619999993593 90
1131 0 1 0 1 0 220.07820000010543 45
1132 1 0 0 0 0 0 180
1133 0 1 0 1 0 191.9794000000693 270
1134 1 0 0 0 0 0 225
1135 0 1 0 1 0 271.97460000007413 180
1136 0 1 0 1 0 224.01029999996535 315


In [162]:
sum(catch)

7